# Navier Stokes - flow around cylinder

## Formulation

In this example we will show how to compute the flow of a Newtonian fluid around a cylinder. The following figure shows the computational mesh used. 
![Mesh](fig/hron_turek_bench.png "Mesh")
The strong formulation of the time dependent Navier Stokes equations for this problem is as following
\begin{align}
\frac{\partial\mathbf{u}}{\partial t} + \mathbf{u}\cdot\nabla\mathbf{u} + \nabla p - \nu\Delta\mathbf{u} &= 0\quad\text{in }\,\Omega\\
\nabla\cdot\mathbf{u}&=0\quad\text{in }\,\Omega\\
\mathbf{u} &= \mathbf{u}_{\text{in}}\quad\text{on }\,\Gamma_{\text{in}}\\
\mathbf{u} &= \mathbf{0}\quad\text{on }\,\Gamma_{\text{wall}}\cup\Gamma_{\text{cylinder}}\\
(-p \mathbb{I} + \nu \nabla \mathbf{u}) \mathbf{n} &= \mathbf{0}\quad\text{on }\,\Gamma_{\text{out}}
\end{align}


Compared to the Stokes equations, the Navier Stokes equations include a convection term $\mathbf{u}\cdot\nabla\mathbf{u}$. Therefore, a nonlinear solver has to be used instead of a linear solver. Another new term is the time derivative $\frac{\partial \mathbf{u}}{\partial t}$ which we will treat using a Crank-Nicholson time stepping scheme.

TODO: Crank Nicholson

## Implementation
First, we need to import all neccessary packages.

In [120]:
import dolfin as df
from time import time

Then we read mesh and boundary markers from the file.

In [112]:
mesh = df.Mesh()
hdf = df.HDF5File(mesh.mpi_comm(), "bench_csg.h5", "r")
hdf.read(mesh, "/mesh", False)

bndry = df.MeshFunction("size_t", mesh, mesh.topology().dim()-1, 0)
hdf.read(bndry, "/boundaries")

with df.XDMFFile("results/boundaries.xdmf") as xdmf:
    xdmf.write(bndry)

Next, we define the function spaces. The MINI element is used in this example.

In [121]:
dim = mesh.geometry().dim()
U = df.FiniteElement("CG", mesh.ufl_cell(), 1)
B = df.FiniteElement("Bubble", mesh.ufl_cell(), dim + 1)
P = df.FiniteElement("CG", mesh.ufl_cell(), 1)
V = df.VectorElement(df.NodalEnrichedElement(U, B))
W = df.FunctionSpace(mesh, df.MixedElement([V, P]))

Let us define some parameters and constants.

In [123]:
velocity = 1.5
nu = df.Constant(0.001)
theta = df.Constant(0.5)
dt = 0.1
t_end = 15

Let us define the boundary conditions. The boundary markings are inlet: 1, outlet: 2, wall: 3, cylinder: 5.

In [115]:
inlet_expression = df.Expression(("v * 4.0 * x[1] * (D - x[1]) / pow(D,2)", "0.0"), D=0.41, v=velocity, degree=2)
bc_in = df.DirichletBC(W.sub(0), inlet_expression, bndry, 1)

zero_vector = df.Constant((0.0, 0.0))
bc_walls = df.DirichletBC(W.sub(0), zero_vector, bndry, 3)
bc_sphere = df.DirichletBC(W.sub(0), zero_vector, bndry, 5)

bcs = [bc_in, bc_walls, bc_sphere]

Now we define the variational problem.

In [116]:
v, q = df.TestFunctions(W)
w = df.Function(W)
w0 = df.Function(W)
u, p = df.split(w)
u0, p0 = df.split(w0)

a = lambda u, v: df.inner(df.grad(u)*u, v)*df.dx + nu*df.inner(df.grad(u), df.grad(v))*df.dx
b = lambda q, v: q*df.div(v)*df.dx

F1 = a(u, v) - b(p, v) - b(q, u)
F0 = a(u0, v) - b(p, v) - b(q, u)

# time derivative
F = df.Constant(1.0/dt)*df.inner((u-u0), v)*df.dx + theta*F1 + (1-theta)*F0

# Jacobi matrix
J = df.derivative(F, w)

Setting up solver for nonlinear variational problem.

In [117]:
problem = df.NonlinearVariationalProblem(F, w, bcs, J)
solver = df.NonlinearVariationalSolver(problem)
solver.parameters['newton_solver']['linear_solver'] = 'mumps'
solver.parameters['newton_solver']['absolute_tolerance'] = 1e-12
solver.parameters['newton_solver']['relative_tolerance'] = 1e-12

Setup files to save results for viewing in Paraview.

In [118]:
ufile = df.XDMFFile("results/u.xdmf")
pfile = df.XDMFFile("results/p.xdmf")
ufile.parameters["flush_output"] = True
pfile.parameters["flush_output"] = True

Time stepping

In [119]:
tick = time()
t = 0.0
(u, p) = w.split(True)
u.rename("v", "velocity")
p.rename("p", "pressure")
df.assign(u, w.sub(0))
df.assign(p, w.sub(1))
ufile.write(u, t)
pfile.write(p, t)
while t < t_end:
    w0.assign(w)
    t += dt
    print("t={:.1f}s".format(t))
    solver.solve()
    df.assign(u, w.sub(0))
    df.assign(p, w.sub(1))
    ufile.write(u, t)
    pfile.write(p, t)
print("ellapsed = ", time() - tick, "s")

t=0.1s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.465e+01 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 2.375e-02 (tol = 1.000e-12) r (rel) = 1.621e-03 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 2.574e-03 (tol = 1.000e-12) r (rel) = 1.757e-04 (tol = 1.000e-12)
  Newton iteration 3: r (abs) = 8.263e-05 (tol = 1.000e-12) r (rel) = 5.642e-06 (tol = 1.000e-12)
  Newton iteration 4: r (abs) = 1.730e-07 (tol = 1.000e-12) r (rel) = 1.181e-08 (tol = 1.000e-12)
  Newton iteration 5: r (abs) = 5.011e-13 (tol = 1.000e-12) r (rel) = 3.421e-14 (tol = 1.000e-12)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
t=0.2s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.203e-01 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 2.122e-03 (tol = 1.000e-12) r (rel) = 1.764e-02 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 9.660e-05 (tol = 1.

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.681e-03 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.724e-05 (tol = 1.000e-12) r (rel) = 1.026e-02 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 8.700e-10 (tol = 1.000e-12) r (rel) = 5.175e-07 (tol = 1.000e-12)
  Newton iteration 3: r (abs) = 1.457e-16 (tol = 1.000e-12) r (rel) = 8.667e-14 (tol = 1.000e-12)
  Newton solver finished in 3 iterations and 3 linear solver iterations.
t=1.7s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.510e-03 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.271e-05 (tol = 1.000e-12) r (rel) = 8.416e-03 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 4.325e-10 (tol = 1.000e-12) r (rel) = 2.865e-07 (tol = 1.000e-12)
  Newton iteration 3: r (abs) = 1.413e-16 (tol = 1.000e-12) r (rel) = 9.356e-14 (tol = 1.000e-12)
  Newton solver finished in 3 iterations and 3 linear solv

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.732e-04 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 2.322e-07 (tol = 1.000e-12) r (rel) = 8.497e-04 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 1.133e-13 (tol = 1.000e-12) r (rel) = 4.147e-10 (tol = 1.000e-12)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
t=3.4s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.496e-04 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.776e-07 (tol = 1.000e-12) r (rel) = 7.114e-04 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 5.949e-14 (tol = 1.000e-12) r (rel) = 2.383e-10 (tol = 1.000e-12)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
t=3.5s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.310e-04 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.421

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.595e-05 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.599e-09 (tol = 1.000e-12) r (rel) = 6.162e-05 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 1.448e-16 (tol = 1.000e-12) r (rel) = 5.579e-12 (tol = 1.000e-12)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
t=5.6s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.270e-05 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.230e-09 (tol = 1.000e-12) r (rel) = 5.418e-05 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 1.407e-16 (tol = 1.000e-12) r (rel) = 6.198e-12 (tol = 1.000e-12)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
t=5.7s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.984e-05 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 9.439

  Newton iteration 0: r (abs) = 1.733e-06 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 7.728e-12 (tol = 1.000e-12) r (rel) = 4.459e-06 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 1.426e-16 (tol = 1.000e-12) r (rel) = 8.228e-11 (tol = 1.000e-12)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
t=7.6s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.515e-06 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 5.971e-12 (tol = 1.000e-12) r (rel) = 3.941e-06 (tol = 1.000e-12)
  Newton iteration 2: r (abs) = 1.425e-16 (tol = 1.000e-12) r (rel) = 9.407e-11 (tol = 1.000e-12)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
t=7.7s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.324e-06 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 4.621e-12 (tol = 1.000e-12) r (rel) = 3.489e

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 7.901e-08 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 2.502e-14 (tol = 1.000e-12) r (rel) = 3.166e-07 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=10.0s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 7.008e-08 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.999e-14 (tol = 1.000e-12) r (rel) = 2.852e-07 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=10.1s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 6.221e-08 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.599e-14 (tol = 1.000e-12) r (rel) = 2.570e-07 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=10.2s
Solving nonlinear variational problem.
t=10.3s
  New

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 4.326e-09 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.687e-16 (tol = 1.000e-12) r (rel) = 3.900e-08 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=12.6s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 3.892e-09 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.614e-16 (tol = 1.000e-12) r (rel) = 4.148e-08 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=12.7s
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 3.502e-09 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.556e-16 (tol = 1.000e-12) r (rel) = 4.442e-08 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
t=12.8s
Solving nonlinear variational problem.
  Newton iter

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.888e-10 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
  Newton iteration 1: r (abs) = 1.425e-16 (tol = 1.000e-12) r (rel) = 4.935e-07 (tol = 1.000e-12)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
ellapsed =  65.5371572971344 s


## Complete Code

Run the code using the command ```python3 NavierStokes2D.py```.

In [108]:
#TODO: copy complete code